## 웹 스크래핑을 이용한 데이터 수집

* open API를 이용한 데이터 수집이 불가한 경우 해당
* 웹을 요청하여 html을 모두 받아서 DOM으로 올려서 데이터를 추출하는 방식
* DOM을 띄워서 데이터를 추출할 때 사용하는 라이브러리
> beautifulsoup(bs4)를 활용  
> conda install beautifulsoup4

In [2]:
from bs4 import BeautifulSoup

In [3]:
from urllib.request import urlopen

- 콘텐츠가 존재하는 해당 페이지까지 진입
- 진입 중에 로그인, ajax 등 사람의 손을 타지 않는지 체크
- 그냥 url만 넣으면 화면이 구성된다 -> OK
- html 자체에 프레임이 적용된 경우, 실제 주소까지 찾아서 이동
- 통신 시 get, post 등 데이터를 전달해서 획득하는 것도 OK

### 네이버 금융 > 고시환율정보 수집

- 하루에 환율이 수시로 변경된다
- 그 주기를 관찰하여 수집 주기를 결정해야 한다

In [4]:
target_site = 'https://finance.naver.com/marketindex/exchangeList.nhn'
target_site

'https://finance.naver.com/marketindex/exchangeList.nhn'

In [5]:
# 사이트로 진입
page = urlopen( target_site )
page

In [6]:
# DOM 구성
# 'html5lib' 파서는 html 양이 크거나, 정교한 파싱을 할 때
# 즉, 아래 'html.parser' 파서가 정상적으로 결과를 내지 못하면 이 파서로 교체
soup = BeautifulSoup( page, 'html.parser' )
# soup

In [7]:
# soup.find_all('td', 'tit')        # [] : 리스트

In [8]:
# 리스트 -> 반복문으로 데이터 추출
for td in soup.find_all('td', 'tit') :
    print(td.a.string.strip())

미국 USD
유럽연합 EUR
일본 JPY (100엔)
중국 CNY
홍콩 HKD
대만 TWD
영국 GBP
오만 OMR
캐나다 CAD
스위스 CHF
스웨덴 SEK
호주 AUD
뉴질랜드 NZD
체코 CZK
칠레 CLP
터키 TRY
몽골 MNT
이스라엘 ILS
덴마크 DKK
노르웨이 NOK
사우디아라비아 SAR
쿠웨이트 KWD
바레인 BHD
아랍에미리트 AED
요르단 JOD
이집트 EGP
태국 THB
싱가포르 SGD
말레이시아 MYR
인도네시아 IDR 100
카타르 QAR
카자흐스탄 KZT
브루나이 BND
인도 INR
파키스탄 PKR
방글라데시 BDT
필리핀 PHP
멕시코 MXN
브라질 BRL
베트남 VND 100
남아프리카 공화국 ZAR
러시아 RUB
헝가리 HUF
폴란드 PLN


In [9]:
# 리스트 내포로 간단하게 가공
tmp = [td.a.string.strip() for td in soup.find_all('td', 'tit')]

In [10]:
# 슬라이싱
tmp[:3]

['미국 USD', '유럽연합 EUR', '일본 JPY (100엔)']

- 데이터를 추출하여 DB에 적재하기 위해 최종 형태는 다음과 같다  
[
    {
        'name' : '미국 USD',
        'code' : 'USD',
        'buy_std_rate' : 1209.90
        'cash_sell' : 1188.73
    }
]

In [11]:
# 찾고자 하는 요소에서 오른쪽 마우스 > Copy > Copy selector
# body > div > table > tbody > tr:nth-child(1) > td:nth-child(4)

- 만약 대상 페이지에 table이 여러 개 존재한다면 특정해서 찾아야 한다
- 대상 데이터가 있는 tr을 모두 찾았다
- table.tbl_exchange > tbody > tr
- 위의 표현을 for문으로 구동하고 하나하나의 tr에서 td:nth-child(4)를 선택

> https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [12]:
soup.select('table.tbl_exchange > tbody > tr')  # 리스트

[<tr>
 <td class="tit"><a href="/marketindex/exchangeDetail.nhn?marketindexCd=FX_USDKRW" onclick="parent.clickcr(this, 'exl.exlist', 'FX_USDKRW', '1', event);" target="_parent">
 				
 					
 					
 					
 					미국 USD
 				
 				</a></td>
 <td class="sale">1,211.00</td>
 <td>1,232.19</td>
 <td>1,189.81</td>
 <td>1,222.80</td>
 <td>1,199.20</td>
 <td>1.000</td>
 </tr>, <tr>
 <td class="tit"><a href="/marketindex/exchangeDetail.nhn?marketindexCd=FX_EURKRW" onclick="parent.clickcr(this, 'exl.exlist', 'FX_EURKRW', '2', event);" target="_parent">
 				
 					
 					
 					
 					유럽연합 EUR
 				
 				</a></td>
 <td class="sale">1,355.71</td>
 <td>1,382.68</td>
 <td>1,328.74</td>
 <td>1,369.26</td>
 <td>1,342.16</td>
 <td>1.120</td>
 </tr>, <tr>
 <td class="tit"><a href="/marketindex/exchangeDetail.nhn?marketindexCd=FX_JPYKRW" onclick="parent.clickcr(this, 'exl.exlist', 'FX_JPYKRW', '3', event);" target="_parent">
 				
 					일본 JPY (100엔)
 					
 					
 					
 				
 				</a></td>
 <td class="

In [19]:
# 원하는 데이터를 감싸는 tr을 찾아서 그 밑에서 자식들을 탐색하고
# 그 자식들에서 데이터를 추출
for tr in soup.select('table.tbl_exchange > tbody > tr') :
    # 데이터를 한 줄씩 뽑아서 -> tr에서 탐색하여 세부 데이터 추출
    print(tr.select_one('td.tit').string.strip())
    print(tr.select_one('td.sale').string.strip())
    print(tr.select('td')[3].string.strip())
    # print(tr.select_one('td:nth-child(4)').string.strip())

미국 USD
1,211.00
1,189.81
유럽연합 EUR
1,355.71
1,328.74
일본 JPY (100엔)
1,143.42
1,123.42
중국 CNY
171.00
162.45
홍콩 HKD
154.46
151.42
대만 TWD
38.67
35.97
영국 GBP
1,467.43
1,438.53
오만 OMR
3,145.37
2,956.65
캐나다 CAD
915.83
897.79
스위스 CHF
1,243.07
1,218.59
스웨덴 SEK
126.45
123.36
호주 AUD
824.15
807.92
뉴질랜드 NZD
785.21
769.75
체코 CZK
52.50
47.78
칠레 CLP
1.71
1.58
터키 TRY
219.96
0.00
몽골 MNT
0.45
0.00
이스라엘 ILS
347.91
320.08
덴마크 DKK
181.65
177.20
노르웨이 NOK
135.77
132.45
사우디아라비아 SAR
322.85
300.58
쿠웨이트 KWD
3,981.85
3,663.31
바레인 BHD
3,212.12
2,955.16
아랍에미리트 AED
329.71
306.97
요르단 JOD
1,708.04
1,571.40
이집트 EGP
73.11
0.00
태국 THB
39.40
37.04
싱가포르 SGD
875.95
858.52
말레이시아 MYR
289.40
267.99
인도네시아 IDR 100
8.56
7.71
카타르 QAR
332.60
0.00
카자흐스탄 KZT
3.12
0.00
브루나이 BND
875.95
823.40
인도 INR
17.13
0.00
파키스탄 PKR
7.69
0.00
방글라데시 BDT
14.35
0.00
필리핀 PHP
23.33
21.42
멕시코 MXN
62.20
56.98
브라질 BRL
309.00
284.28
베트남 VND 100
5.22
4.61
남아프리카 공화국 ZAR
79.89
73.50
러시아 RUB
18.54
16.51
헝가리 HUF
4.18
3.85
폴란드 PLN
313.59
288.51


In [21]:
results = []
for tr in soup.select('table.tbl_exchange > tbody > tr') :
    tmp = dict()
    tmp['name'] = tr.select_one('td.tit').string.strip()
    tmp['code'] = tr.select_one('td.tit').a['href'][-6:-3]    # 태그[속성]
    tmp['buy_std_rate'] = tr.select_one('td.sale').string.strip()
    tmp['cash_sell'] = tr.select('td')[3].string.strip()
    results.append(tmp) 

In [22]:
results

[{'name': '미국 USD',
  'code': 'USD',
  'buy_std_rate': '1,211.00',
  'cash_sell': '1,189.81'},
 {'name': '유럽연합 EUR',
  'code': 'EUR',
  'buy_std_rate': '1,355.71',
  'cash_sell': '1,328.74'},
 {'name': '일본 JPY (100엔)',
  'code': 'JPY',
  'buy_std_rate': '1,143.42',
  'cash_sell': '1,123.42'},
 {'name': '중국 CNY',
  'code': 'CNY',
  'buy_std_rate': '171.00',
  'cash_sell': '162.45'},
 {'name': '홍콩 HKD',
  'code': 'HKD',
  'buy_std_rate': '154.46',
  'cash_sell': '151.42'},
 {'name': '대만 TWD',
  'code': 'TWD',
  'buy_std_rate': '38.67',
  'cash_sell': '35.97'},
 {'name': '영국 GBP',
  'code': 'GBP',
  'buy_std_rate': '1,467.43',
  'cash_sell': '1,438.53'},
 {'name': '오만 OMR',
  'code': 'OMR',
  'buy_std_rate': '3,145.37',
  'cash_sell': '2,956.65'},
 {'name': '캐나다 CAD',
  'code': 'CAD',
  'buy_std_rate': '915.83',
  'cash_sell': '897.79'},
 {'name': '스위스 CHF',
  'code': 'CHF',
  'buy_std_rate': '1,243.07',
  'cash_sell': '1,218.59'},
 {'name': '스웨덴 SEK',
  'code': 'SEK',
  'buy_std_rate': '

In [23]:
# 리스트 내포
[{
    'name' : tr.select_one('td.tit').string.strip(),
    'code' : tr.select_one('td.tit').a['href'][-6:-3],
    'buy_std_rate' : tr.select_one('td.sale').string.strip(),
    'cash_sell' : tr.select('td')[3].string.strip()
} for tr in soup.select('table.tbl_exchange > tbody > tr') ]

[{'name': '미국 USD',
  'code': 'USD',
  'buy_std_rate': '1,211.00',
  'cash_sell': '1,189.81'},
 {'name': '유럽연합 EUR',
  'code': 'EUR',
  'buy_std_rate': '1,355.71',
  'cash_sell': '1,328.74'},
 {'name': '일본 JPY (100엔)',
  'code': 'JPY',
  'buy_std_rate': '1,143.42',
  'cash_sell': '1,123.42'},
 {'name': '중국 CNY',
  'code': 'CNY',
  'buy_std_rate': '171.00',
  'cash_sell': '162.45'},
 {'name': '홍콩 HKD',
  'code': 'HKD',
  'buy_std_rate': '154.46',
  'cash_sell': '151.42'},
 {'name': '대만 TWD',
  'code': 'TWD',
  'buy_std_rate': '38.67',
  'cash_sell': '35.97'},
 {'name': '영국 GBP',
  'code': 'GBP',
  'buy_std_rate': '1,467.43',
  'cash_sell': '1,438.53'},
 {'name': '오만 OMR',
  'code': 'OMR',
  'buy_std_rate': '3,145.37',
  'cash_sell': '2,956.65'},
 {'name': '캐나다 CAD',
  'code': 'CAD',
  'buy_std_rate': '915.83',
  'cash_sell': '897.79'},
 {'name': '스위스 CHF',
  'code': 'CHF',
  'buy_std_rate': '1,243.07',
  'cash_sell': '1,218.59'},
 {'name': '스웨덴 SEK',
  'code': 'SEK',
  'buy_std_rate': '

In [25]:
# 일본 제외 출력
results_withoutJapan = []
for tr in soup.select('table.tbl_exchange > tbody > tr') :
    if tr.select_one('td.tit').string.strip().count('JPY') == 0 :
        tmp = dict()
        tmp['name'] = tr.select_one('td.tit').string.strip()
        tmp['code'] = tr.select_one('td.tit').a['href'][-6:-3]
        tmp['buy_std_rate'] = tr.select_one('td.sale').string.strip()
        tmp['cash_sell'] = tr.select('td')[3].string.strip()
        results_withoutJapan.append(tmp)
results_withoutJapan

[{'name': '미국 USD',
  'code': 'USD',
  'buy_std_rate': '1,211.00',
  'cash_sell': '1,189.81'},
 {'name': '유럽연합 EUR',
  'code': 'EUR',
  'buy_std_rate': '1,355.71',
  'cash_sell': '1,328.74'},
 {'name': '중국 CNY',
  'code': 'CNY',
  'buy_std_rate': '171.00',
  'cash_sell': '162.45'},
 {'name': '홍콩 HKD',
  'code': 'HKD',
  'buy_std_rate': '154.46',
  'cash_sell': '151.42'},
 {'name': '대만 TWD',
  'code': 'TWD',
  'buy_std_rate': '38.67',
  'cash_sell': '35.97'},
 {'name': '영국 GBP',
  'code': 'GBP',
  'buy_std_rate': '1,467.43',
  'cash_sell': '1,438.53'},
 {'name': '오만 OMR',
  'code': 'OMR',
  'buy_std_rate': '3,145.37',
  'cash_sell': '2,956.65'},
 {'name': '캐나다 CAD',
  'code': 'CAD',
  'buy_std_rate': '915.83',
  'cash_sell': '897.79'},
 {'name': '스위스 CHF',
  'code': 'CHF',
  'buy_std_rate': '1,243.07',
  'cash_sell': '1,218.59'},
 {'name': '스웨덴 SEK',
  'code': 'SEK',
  'buy_std_rate': '126.45',
  'cash_sell': '123.36'},
 {'name': '호주 AUD',
  'code': 'AUD',
  'buy_std_rate': '824.15',
  

In [26]:
# 일본 제외 리스트 내포
[{
    'name' : tr.select_one('td.tit').string.strip(),
    'code' : tr.select_one('td.tit').a['href'][-6:-3],
    'buy_std_rate' : tr.select_one('td.sale').string.strip(),
    'cash_sell' : tr.select('td')[3].string.strip()
} for tr in soup.select('table.tbl_exchange > tbody > tr')
  if tr.select_one('td.tit').string.strip().count('JPY') == 0 ]

[{'name': '미국 USD',
  'code': 'USD',
  'buy_std_rate': '1,211.00',
  'cash_sell': '1,189.81'},
 {'name': '유럽연합 EUR',
  'code': 'EUR',
  'buy_std_rate': '1,355.71',
  'cash_sell': '1,328.74'},
 {'name': '중국 CNY',
  'code': 'CNY',
  'buy_std_rate': '171.00',
  'cash_sell': '162.45'},
 {'name': '홍콩 HKD',
  'code': 'HKD',
  'buy_std_rate': '154.46',
  'cash_sell': '151.42'},
 {'name': '대만 TWD',
  'code': 'TWD',
  'buy_std_rate': '38.67',
  'cash_sell': '35.97'},
 {'name': '영국 GBP',
  'code': 'GBP',
  'buy_std_rate': '1,467.43',
  'cash_sell': '1,438.53'},
 {'name': '오만 OMR',
  'code': 'OMR',
  'buy_std_rate': '3,145.37',
  'cash_sell': '2,956.65'},
 {'name': '캐나다 CAD',
  'code': 'CAD',
  'buy_std_rate': '915.83',
  'cash_sell': '897.79'},
 {'name': '스위스 CHF',
  'code': 'CHF',
  'buy_std_rate': '1,243.07',
  'cash_sell': '1,218.59'},
 {'name': '스웨덴 SEK',
  'code': 'SEK',
  'buy_std_rate': '126.45',
  'cash_sell': '123.36'},
 {'name': '호주 AUD',
  'code': 'AUD',
  'buy_std_rate': '824.15',
  